In [ ]:
import pandas as pd
import pymysql
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler


df = pd.read_csv("venta.csv")

df = df.rename(columns={
    "precio": "precio",
    "titulo": "titulo",
    "ubicacion": "ubicacion",
    "habitacionnum": "habitaciones",
    "baños": "banos",
    "superficie_construida": "superficie_construida",
    "superficie_util": "superficie_util",
    "descripcion_ampliada": "descripcion_ampliada",
    "tipo_vivienda": "tipo_vivienda",
    "link": "link",
    "precio_m2": "precio_m2"
})

df["metros"] = df["superficie_construida"]

features = ["precio", "metros", "habitaciones", "banos"]
for col in features:
    if col not in df.columns:
        raise ValueError(f"La columna '{col}' no está en el DataFrame")

df_filtrado = df[features].dropna()
if len(df_filtrado) < 3:
    raise ValueError("No hay suficientes datos completos para aplicar clustering.")

escalador = StandardScaler()
X = escalador.fit_transform(df_filtrado)
modelo = KMeans(n_clusters=3, random_state=42, n_init=10)
df["cluster"] = None
df.loc[df_filtrado.index, "cluster"] = modelo.fit_predict(X)

conexion = pymysql.connect(
    host="localhost",
    user="root",
    password="Destroyerl0",
    database="propiedades_de_ventas",
    charset="utf8mb4"
)
cursor = conexion.cursor()

for _, fila in df.iterrows():
    sql = """
    INSERT INTO anuncios (
        titulo, precio, ubicacion, habitaciones, banos, metros,
        superficie_construida, superficie_util, tipo_vivienda,
        descripcion_ampliada, link, precio_m2
    ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """
    datos = (
        fila["titulo"],
        float(fila["precio"]) if pd.notna(fila["precio"]) else None,
        fila["ubicacion"],
        int(fila["habitaciones"]) if pd.notna(fila["habitaciones"]) else None,
        int(fila["banos"]) if pd.notna(fila["banos"]) else None,
        int(fila["metros"]) if pd.notna(fila["metros"]) else None,
        int(fila["superficie_construida"]) if pd.notna(fila["superficie_construida"]) else None,
        int(fila["superficie_util"]) if pd.notna(fila["superficie_util"]) else None,
        fila["tipo_vivienda"],
        fila["descripcion_ampliada"],
        fila["link"],
        float(fila["precio_m2"]) if pd.notna(fila["precio_m2"]) else "datos insuficientes"
    )
    cursor.execute(sql, datos)

conexion.commit()
cursor.close()
conexion.close()